In [ ]:
!pip install langchain_openai langchain_core langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
response = llm.invoke("Qual a previsão do tempo em Recife-RN?")
print(response.content)

Desculpe, mas não consigo fornecer informações em tempo real, como a previsão do tempo. Recomendo que você consulte um site de meteorologia ou um aplicativo de clima para obter as informações mais atualizadas sobre o tempo em Recife-RN.


In [ ]:
a = 18273840567
b = 12830192835

response = llm.invoke("Quanto é 18273840567*12830192835? Responda apenas a o resultado.")
print(response.content)
print()
print(f'Resposta esperada:', a*b)

234084204189203999595.

Resposta esperada: 234456898310655737445


In [ ]:
import random
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Soma dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a * b

@tool
def get_weather(city: str) -> str:
    """Retorna um clima aleatório para uma cidade.

    Args:
        city: Nome da cidade
    """
    weather = random.choice(["ensolarado", "nublado", "chuvoso", "tempestuoso", "neve"])
    return weather

@tool
def write_file(filename: str, content: str):
    """Função para criar arquivos de texto. Retorna um arquivo txt

       Args:
        filename: Nome do arquivo
        content: Conteúdo do arquivo
    """
    with open(filename, "w") as f:
        f.write(content)
    return filename

@tool
def letter_count(word: str, letter: str) -> int:
    """Conta a quantidade de ocorrências de uma letra em uma palavra.

    Args:
        word: Palavra a ser contada
        letter: Letra a ser contada
    """
    word = word.lower()
    letter = letter.lower()
    return word.count(letter)


tools = [add, multiply, get_weather, write_file, letter_count]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
get_weather.invoke({"city": "Recife-RN"})

'ensolarado'

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")]

ai_msg = llm_with_tools.invoke(messages)

print(f"ContentString: {ai_msg.content}")
print(f"ToolCalls: {ai_msg.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'get_weather', 'args': {'city': 'Recife-RN'}, 'id': 'call_JLNFtpRgsoSQp1pqhrbPmr7e', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 123, 'b': 321}, 'id': 'call_wNaad5lqv8HfLAMJC7Y53GOv', 'type': 'tool_call'}]


In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [ ]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")
    ]
})

print(response["messages"])

[HumanMessage(content='Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?', additional_kwargs={}, response_metadata={}, id='4f12d74c-9688-4ed8-aef1-06341f667c25'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X40P4rVWisXkUi4fTZ1FAXyt', 'function': {'arguments': '{"city": "Recife-RN"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_TNr5J4uXE078qI399koTwptb', 'function': {'arguments': '{"a": 123, "b": 321}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 145, 'total_tokens': 195, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c6e4960f-8d98-4d35-9c6e-0bc8eff0

In [ ]:
for msg in response["messages"]:
    print(msg.type)
    print(msg.content)
    print()

human
Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?

ai


tool
tempestuoso

tool
39483

ai
A previsão do tempo em Recife-RN é de tempo tempestuoso. 

Além disso, o resultado de 123 * 321 é 39.483.



In [ ]:
print(response["messages"][-1].content)

A previsão do tempo em Recife-RN é de tempo tempestuoso. 

Além disso, o resultado de 123 * 321 é 39.483.


## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [ ]:
agent_executor = create_react_agent(llm, tools)

In [ ]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Escreva um conteúdo sobre Java, de até 2 parágrafos e chame o arquivo de aboutjava.txt")
    ]
})

In [ ]:
conteudo = response["messages"][-1].content
print(conteudo)

O arquivo "aboutjava.txt" foi criado com sucesso e contém um conteúdo sobre Java. Se precisar de mais alguma coisa, é só avisar!


### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [ ]:
response_2 = agent_executor.invoke({
    "messages": [
        HumanMessage("Quantas vezes a letra p aparece em paralelepípedo?")
    ]
})

In [ ]:
print(response_2["messages"][-1].content)

A letra "p" aparece 3 vezes na palavra "paralelepípedo".


In [ ]:
letter_count.invoke({"word": "paralelepípedo", "letter": "p"})

3